A combination of twittes dataset and reddit dataset analysis. For the reddit dataset I check for sarcastic words and phrases. For the twittes dataset I extract the 20 most common emojis in sarcastic and non sarcastic comments.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import set_config
import pandas as pd
import numpy as np
import json
set_config(display="diagram")

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

reddit_downloaded = drive.CreateFile({'id':"1U0pQDPYFG1m4ajKU_knjZCQmDMh9jQBp"})
twitter_downloaded = drive.CreateFile({'id':"1sxYXc7gy1ZaYW-L0geDXunEMZ3h9ixSl"})
reddit_downloaded.GetContentFile('train-balanced-sarcasm.csv')
twitter_downloaded.GetContentFile('twittes.json')
reddit_data = pd.read_csv('train-balanced-sarcasm.csv')
reddit_data['comment'] = reddit_data['comment'].dropna()
reddit_data['label'] = reddit_data['label'].dropna()

# Opening JSON file
f = open('twittes.json')
twitter_data = json.load(f)
f.close()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
sarcastic_words = ["obviously", "clearly", "totally", "duh", "everyone knows", "right because", "yeah obviously", "yes because", "yeah because","shitlord",
                    " definitely ", "of course", "surely", "how dare", "duh", " gee ", "for sure", "amirite", "good thing that","how i love", "what a surprise"]


def get_features(comment):
  return np.array([1 if x in comment else 0 for x in sarcastic_words])

In [ ]:
X = []
Y = []
comments = []

for x,y in zip(reddit_data['comment'].tolist(), reddit_data['label'].tolist()):
  if type(x) is str and len(x) < 150:
    comments.append(x.lower().replace(',', ''))
    X.append(get_features(x.lower().replace(',', '')))
    Y.append(y)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state=42)
lgr = LogisticRegression(random_state=0,max_iter=500).fit(X_train, Y_train)

Y_pred = lgr.predict(X)

In [ ]:
count_1 = 0
for i in Y_pred:
  if i == '1':
     count_1 += 1

print(count_1)
print(count_1 / len(Y_pred))

report = classification_report(Y_pred,Y, output_dict=True)
report = pd.DataFrame(report).transpose()
report = report.rename({"0": "Sarcastic Comments", "1": "Non Sarcastic Comments"})
report

47644
0.048890462337289185


,precision,recall,f1-score,support
Sarcastic Comments,0.987772,0.514187,0.676316,926861.000000
Non Sarcastic Comments,0.084841,0.876165,0.154702,47644.000000
accuracy,0.531884,0.531884,0.531884,0.531884
macro avg,0.536306,0.695176,0.415509,974505.000000
weighted avg,0.943627,0.531884,0.650814,974505.000000


Here I check random words for how frequent they are in sarcastic comments vs not sarcastic comments according a linear regression classification model trained on the reddit dataset.   

In [ ]:
count_word = 0
not_word = 0
for i,com in enumerate(comments):
  if "how i love" in com:
    if Y[i] == 1:
      count_word +=1
    else:
      not_word +=1
print("sarcastic:",count_word)
print("not sarcasticL:", not_word)

sarcastic: 17
not sarcasticL: 3


In [ ]:
#twitter_data = twitter_data['_default']
twitter_texts = [get_features(x['data']['fullText'].lower().replace(',', '')) for x in twitter_data.values()]
pred_tweets = lgr.predict(twitter_texts)
for k in twitter_data.keys():
  twitter_data[k]['data']['pred'] = pred_tweets[int(k)-1]


In [ ]:
count_1 = 0
sarc_tweets = []
for v in twitter_data.values():
  if v['data']['pred'] == '1':
    count_1 += 1
    sarc_tweets += [v['data']['fullText']]

print(count_1)
print(count_1 / len(Y_pred))
sarc_tweets[:20]

4083
0.016759226194141042


['@PeterStefanovi2 @LondonItl Boris Johnson lying? Surely not! 🙄 #sarcasm',
 'I was very upset that this year\'s #Bojo #Christmas "Business party" was called off.....I mean... C\'mon....Rules for #plebs don\'t apply to "Business" ...duh! #sarcasm #whatcovid 🤣 https://t.co/D4uczgA81f',
 'RT @XRPmelch: @AussieVal10 "as we move through the next phase"\n\nYeah because that doesn\'t sound planned at all 🤔 #sarcasm\n\nTime to wake up…',
 '@AussieVal10 "as we move through the next phase"\n\nYeah because that doesn\'t sound planned at all 🤔 #sarcasm\n\nTime to wake up and say NO.\nThat\'s the only way this ends.',
 '#DecoupleD is actually something different concept and yes obviously an entertaining content\n\nIf this kind of #Sarcasm\nAny couple have in-between\nThen\nwhy would they choose to be separated😛😂\n\nLiked it❤️\n @ActorMadhavan Sir👏👏🙌\n@SurveenChawla ✌️\n\n#DecoupledReview\n#netflixindia',
 'RT @PhoenixWolfJ: @APFactCheck @AP Of course. ...  Cause Trudeau wouldnt violate Canadians F

In [ ]:
sarc_emojis = []
not_sarc_emojis = []
for v in twitter_data.values():
  if v['data']['pred'] == '1':
    sarc_emojis += v['data']['emojis']
  else:
    not_sarc_emojis += v['data']['emojis']

In [ ]:
from collections import Counter
c1 = Counter(sarc_emojis)
c2 = Counter(not_sarc_emojis)

print ("",c1.most_common(10))
print ("",c2.most_common(10))

 [(':face_with_rolling_eyes:', 923), (':face_with_tears_of_joy:', 693), (':rolling_on_the_floor_laughing:', 295), (':winking_face:', 199), (':thinking_face:', 190), (':unamused_face:', 160), (':grinning_face_with_big_eyes:', 135), (':smirking_face:', 86), (':hundred_points:', 79), (':beaming_face_with_smiling_eyes:', 78)]
 [(':face_with_tears_of_joy:', 21335), (':face_with_rolling_eyes:', 9588), (':rolling_on_the_floor_laughing:', 7831), (':thinking_face:', 4402), (':winking_face:', 3472), (':red_heart:', 2780), (':unamused_face:', 2734), (':winking_face_with_tongue:', 2631), (':beaming_face_with_smiling_eyes:', 2541), (':grinning_squinting_face:', 2001)]
